# Machine Learning with Dynamic Time Patterns for Algorithmic Trading

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("usd_dkk_5min.csv", parse_dates=["Time"])
df.set_index("Time", inplace=True)

# Sort the index to ensure monotonicity
df = df.sort_index()

# Define the date range
start_date = "2024.01.02.00:00"
end_date = "2024.01.02.21:25"

# Filter to the first 10 days of 2024
df = df[start_date:end_date].copy()
print(df.tail())


                        Open     High      Low    Close
Time                                                   
2024-01-02 21:05:00  6.80745  6.80842  6.80739  6.80832
2024-01-02 21:10:00  6.80832  6.80916  6.80830  6.80883
2024-01-02 21:15:00  6.80882  6.81001  6.80851  6.80858
2024-01-02 21:20:00  6.80858  6.80945  6.80844  6.80932
2024-01-02 21:25:00  6.80933  6.80976  6.80896  6.80960


In [5]:
# Calculate log returns
df["returns"] = 1+np.log(df["Close"] / df["Close"].shift(1))
df.dropna(inplace=True)

In [6]:
from tslearn.metrics import dtw
from sklearn.neighbors import NearestNeighbors
import numpy as np
from fastdtw import fastdtw


def knn_dtw_forecast(series, window_size=150, embed_size=10, k=7):
    predictions = []
    
    # FastDTW metric for kNN
    def dtw_metric(x, y):
        x_norm = (x - x.mean()) / x.std()
        y_norm = (y - y.mean()) / y.std()
        return fastdtw(x_norm, y_norm)[0]  # Returns only distance

    # Main loop
    for i in range(window_size + embed_size, len(series)):
        query = series.iloc[i - embed_size:i].values
        history = series.iloc[:i - embed_size].values

        if len(history) < window_size:
            predictions.append(0)
            continue

        # Create rolling window of historical embeddings
        start_index = max(0, len(history) - window_size)
        historical_embeddings = np.array([history[j:j + embed_size] for j in range(start_index, len(history) - embed_size + 1)])

        if query.std() == 0 or np.any(historical_embeddings.std(axis=1) == 0):
            predictions.append(0)
            continue

        # Parallelized kNN search
        nbrs = NearestNeighbors(n_neighbors=k, metric=dtw_metric, n_jobs=-1).fit(historical_embeddings)
        dist, indices = nbrs.kneighbors([query])

        # Predict next return using weighted average
        next_returns = [series.iloc[start_index + idx + embed_size] for idx in indices[0]]
        weights = 1 / (dist[0] + 1e-6)  # Avoid div by zero
        prediction = np.average(next_returns, weights=weights)

        predictions.append(prediction)
        
    # Padding at the start
    return [0] * (window_size + embed_size) + predictions

# Run optimized forecast
df["dtw_pred"] = knn_dtw_forecast(df["returns"])


In [7]:
print(df["returns"])
print(df["dtw_pred"])

Time
2024-01-02 00:05:00    0.999996
2024-01-02 00:10:00    0.999774
2024-01-02 00:15:00    0.999960
2024-01-02 00:20:00    0.999892
2024-01-02 00:25:00    1.000028
                         ...   
2024-01-02 21:05:00    1.000126
2024-01-02 21:10:00    1.000075
2024-01-02 21:15:00    0.999963
2024-01-02 21:20:00    1.000109
2024-01-02 21:25:00    1.000041
Name: returns, Length: 257, dtype: float64
Time
2024-01-02 00:05:00    0.000000
2024-01-02 00:10:00    0.000000
2024-01-02 00:15:00    0.000000
2024-01-02 00:20:00    0.000000
2024-01-02 00:25:00    0.000000
                         ...   
2024-01-02 21:05:00    0.999915
2024-01-02 21:10:00    1.000086
2024-01-02 21:15:00    0.999979
2024-01-02 21:20:00    0.999983
2024-01-02 21:25:00    1.000022
Name: dtw_pred, Length: 257, dtype: float64


In [44]:
def backtest_strategy(df, initial_balance=100000, trade_size=0.02, buy_threshold=1.0002, sell_threshold=0.9998):
    positions = 0  # 1 for long, -1 for short, 0 for neutral
    balance = initial_balance

    for i in range(160, len(df) - 1):
        prev_balance = df['balance'].iloc[i - 1] if i > 160 else initial_balance  # Handle the first iteration

        # Determine position based on prediction
        if df['dtw_pred'].iloc[i] > buy_threshold:
            df.loc[df.index[i], 'position'] = 1
            position_size = prev_balance * trade_size
            pnl = df["returns"].iloc[i + 1] * position_size
            df.loc[df.index[i], 'balance'] = prev_balance + pnl

        elif df['dtw_pred'].iloc[i] < sell_threshold:
            df.loc[df.index[i], 'position'] = -1
            position_size = prev_balance * trade_size
            pnl = df["returns"].iloc[i + 1] * position_size
            df.loc[df.index[i], 'balance'] = prev_balance + pnl
        else:
            df.loc[df.index[i], 'balance'] = prev_balance # Maintain balance if no trade
    df["position"] = positions
    df["balance"]= balance
    return df
backtest_strategy(df)

,Open,High,Low,Close,returns,dtw_pred,balance,position
Time,,,,,,,,
2024-01-02 00:05:00,6.76275,6.76363,6.76248,6.76270,0.999996,0.000000,100000,0
2024-01-02 00:10:00,6.76264,6.76302,6.76108,6.76117,0.999774,0.000000,100000,0
2024-01-02 00:15:00,6.76117,6.76138,6.76027,6.76090,0.999960,0.000000,100000,0
2024-01-02 00:20:00,6.76089,6.76111,6.75964,6.76017,0.999892,0.000000,100000,0
2024-01-02 00:25:00,6.76016,6.76049,6.75923,6.76036,1.000028,0.000000,100000,0
...,...,...,...,...,...,...,...,...
2024-01-02 21:05:00,6.80745,6.80842,6.80739,6.80832,1.000126,0.999915,100000,0
2024-01-02 21:10:00,6.80832,6.80916,6.80830,6.80883,1.000075,1.000086,100000,0
2024-01-02 21:15:00,6.80882,6.81001,6.80851,6.80858,0.999963,0.999979,100000,0


In [46]:
print(df.tail())

                        Open     High      Low    Close   returns  dtw_pred  \
Time                                                                          
2024-01-02 21:05:00  6.80745  6.80842  6.80739  6.80832  1.000126  0.999915   
2024-01-02 21:10:00  6.80832  6.80916  6.80830  6.80883  1.000075  1.000086   
2024-01-02 21:15:00  6.80882  6.81001  6.80851  6.80858  0.999963  0.999979   
2024-01-02 21:20:00  6.80858  6.80945  6.80844  6.80932  1.000109  0.999983   
2024-01-02 21:25:00  6.80933  6.80976  6.80896  6.80960  1.000041  1.000022   

                     balance  position  
Time                                    
2024-01-02 21:05:00   100000         0  
2024-01-02 21:10:00   100000         0  
2024-01-02 21:15:00   100000         0  
2024-01-02 21:20:00   100000         0  
2024-01-02 21:25:00   100000         0  
